## Task 3

In [1]:
import numpy as np
import pandas as pd

import pickle

In [2]:
from task1 import *
from task2 import *

[nltk_data] Downloading package punkt to /Users/ryanl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ryanl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/ryanl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ryanl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Prepare Data

In [3]:
vocab = get_vocab()

vocabulary loaded!


In [4]:
candidate_passages, pid_passage_dict = passage_to_id()

pid-to-passage mapped!


In [5]:
candidate_passages.head()

,qid,pid,query,passage
0,494835,7130104,"sensibilities, definition",This is the definition of RNA along with examp...
1,1128373,7130104,iur definition,This is the definition of RNA along with examp...
2,131843,7130104,definition of a sigmet,This is the definition of RNA along with examp...
3,20455,7130335,ar glasses definition,Best Answer: The AR designation comes from the...
4,719381,7130335,what is ar balance,Best Answer: The AR designation comes from the...


In [6]:
inverted_index = get_inverted_index(vocab, pid_passage_dict)

100%|██████████████████████████████████| 182469/182469 [03:22<00:00, 900.62it/s]


## Passage TF-IDF

Extract the **TF-IDF** vector representations of the passages using the **inverted index** you have constructed. 

Using the **IDF** representation from the **corpus of the passages**, extract the **TF-IDF** representation of the **queries**. 

In [7]:
def get_passage_tf_idf(pid_passage_dict, inverted_index):
    
    # initialization
    passage_tf_idf = {}
    N_doc = len(pid_passage_dict)
    
    for pid, passage in tqdm(pid_passage_dict.items()):
        
        pid_tf_idf = {}
        
        for term in passage:
            
            # term frequency 
            freq = inverted_index[term][pid][0]
            tf = freq / len(passage)
            
            # inverse document frequency 
            n_term = len(inverted_index[term])
            idf = np.log10(N_doc/n_term)
            
            # tf-idf of the current passage
            pid_tf_idf[term] = [tf, idf, tf*idf]
        
        passage_tf_idf[pid] = pid_tf_idf
    
    return passage_tf_idf

In [8]:
passage_tf_idf = get_passage_tf_idf(pid_passage_dict, inverted_index)

100%|████████████████████████████████| 182469/182469 [00:07<00:00, 24474.14it/s]


## Query TF-IDF

In [9]:
def query_to_id():
    
    # tokenize the queries and map to qid
    test_queries = pd.read_csv('test-queries.tsv', sep='\t', header=None, names=['qid','query'])
    cleaned_test_queries = process_passage(test_queries['query'], remove_sw=True, lemma=True)
    qid_query_dict = dict(zip(test_queries['qid'], cleaned_test_queries))
    
    return test_queries, qid_query_dict

In [10]:
test_queries, qid_query_dict = query_to_id()

In [11]:
test_queries.head()

,qid,query
0,1108939,what slows down the flow of blood
1,1112389,"what is the county for grand rapids, mn"
2,792752,what is ruclip
3,1119729,what do you do when you have a nosebleed from ...
4,1105095,where is sugar lake lodge located


In [12]:
terms = [v for k, v in qid_query_dict.items()]
query_vocab = list(set([item for sublist in terms for item in sublist]))

In [13]:
len(query_vocab)

530

In [14]:
query_inverted_index = get_inverted_index(query_vocab, qid_query_dict)

100%|██████████████████████████████████████| 200/200 [00:00<00:00, 88171.20it/s]


In [15]:
len(query_inverted_index)

530

In [16]:
def get_query_tf_idf(qid_query_dict, query_inverted_index, vocab, query_vocab, inverted_index, passage_tf_idf):
    
    # initialization
    query_tf_idf = {}
    N_doc = len(qid_query_dict)
    
    for qid, query in tqdm(qid_query_dict.items()):
        
        qid_tf_idf = {}
        
        for term in query:
            
            # skip terms that are not found in passages
            if term not in vocab:
                continue
            
            if term not in query_vocab:
                continue
            
            # term frequency 
            freq = query_inverted_index[term][qid][0]
            tf = freq / len(query)
            
            # inverse document frequency 
            pid = list(inverted_index[term].keys())[0]
            idf_passage = passage_tf_idf[pid][term][1]
                
            # tf-idf of the current query
            qid_tf_idf[term] = [tf, idf_passage, tf*idf_passage]
            
        query_tf_idf[qid] = qid_tf_idf
        
    return query_tf_idf

In [17]:
query_tf_idf = get_query_tf_idf(qid_query_dict, query_inverted_index, vocab, query_vocab, inverted_index, passage_tf_idf)

100%|███████████████████████████████████████| 200/200 [00:00<00:00, 5149.45it/s]


In [18]:
len(query_tf_idf)

200

## Cosine Similarity

In [19]:
def cosine_similarity(v_query, v_passage):
    
    """
    The input vectors are the TF-IDF representations.
    """
    
    # find common terms
    terms = set(v_query.keys()).intersection(set(v_passage.keys()))
    
    # compute the inner product
    inner_product = 0.0
    for term in terms:
        inner_product += v_query[term][2] * v_passage[term][2]
        
    # compute the denominators
    X = np.linalg.norm([v[2] for k,v in v_query.items()])
    Y = np.linalg.norm([v[2] for k,v in v_passage.items()])
    
    if X == 0 or Y == 0:
        return 0
    else:
        return inner_product / (X * Y)

## Vector Space Model

Use a **basic vector space model** with TF-IDF and cosine similarity to retrieve **at most 100 passages** from within the 1000 candidate passages **for each query** (some queries have fewer candidate passages). 

Store the outcomes in a file named **tfidf.csv**. Each row of tfidf.csv must have the following format: **qid,pid,score** where qid denotes the query’s identifier, pid denotes the passage identifier, and score is their cosine similarity score. Note that there should be **no spaces between commas**. 

The output .csv files should **not have headers**. qid,pid pairs should be **ranked by similarity score in descending order**, i.e. starting with the highest similarity score. 

You should first report the top qid,pid pairs for one qid, then move on to the next, following the **same query order** as in the text-queries.tsv file. Each output .csv file is expected to have **19,290 rows**.

In [20]:
def vector_space_model(test_queries, candidate_passages, query_tf_idf, passage_tf_idf, top=100):
    
    # initialization
    df = pd.DataFrame()
    qids, pids, scores = np.array([]), np.array([]), np.array([])
    
    for qid, query in tqdm(test_queries.values):
        
        qids_i, pids_i, scores_i = np.array([]), np.array([]), np.array([])
        
        for pid in candidate_passages[candidate_passages['qid']==qid]['pid']:
            
            v_query = query_tf_idf[qid]
            v_passage = passage_tf_idf[pid]
            score = cosine_similarity(v_query, v_passage)
            
            qids_i = np.append(qids_i, qid)
            pids_i = np.append(pids_i, pid)
            scores_i = np.append(scores_i, score)
    
        descending_idx = np.argsort(scores_i)[::-1]
        qids_i = qids_i[descending_idx]
        pids_i = pids_i[descending_idx]
        scores_i = scores_i[descending_idx]
        
        qids = np.append(qids, qids_i[:top])
        pids = np.append(pids, pids_i[:top])
        scores = np.append(scores, scores_i[:top])
        
    df['qid'] = qids
    df['pid'] = pids
    df['score'] = scores
    
    df['qid'] = df['qid'].astype(int)
    df['pid'] = df['pid'].astype(int)
        
    return df

In [21]:
df_tf_idf = vector_space_model(test_queries, candidate_passages, query_tf_idf, passage_tf_idf, top=100)

100%|█████████████████████████████████████████| 200/200 [00:04<00:00, 42.04it/s]


In [22]:
df_tf_idf.head()

,qid,pid,score
0,1108939,2846462,0.603323
1,1108939,25753,0.541660
2,1108939,5291621,0.525253
3,1108939,7804766,0.513956
4,1108939,1957452,0.506223


In [23]:
df_tf_idf.tail()

,qid,pid,score
19285,532603,1488181,0.306845
19286,532603,2438261,0.306615
19287,532603,6888387,0.306106
19288,532603,231856,0.305912
19289,532603,7216392,0.305757


In [24]:
len(df_tf_idf)

19290

In [25]:
df_tf_idf.to_csv('tfidf.csv', header=False, sep=',', index=False)

## BM25 Retrieval Model

Use the inverted index to also implement **BM25** while setting **k1 = 1.2, k2 = 100**, and **b = 0.75**. 

Retrieve **at most 100 passages** from within the 1000 candidate passages **for each test query**. Store the outcomes in a file named **bm25.csv**.

In [53]:
def BM25_model(query_params, passage_params, model_params, top=100):
    
    # parameters
    test_queries, qid_query_dict, q_inverted_index = query_params[0], query_params[1], query_params[2]
    candidate_passages, pid_passage_dict, p_inverted_index = passage_params[0], passage_params[1], passage_params[2]
    k1, k2, b = model_params[0], model_params[1], model_params[2]
    
    # pre-compute values
    N = len(pid_passage_dict)
    avdl = np.sum([len(passage) for pid, passage in pid_passage_dict.items()]) / len(pid_passage_dict)
    
    # initialization
    df = pd.DataFrame()
    qids, pids, scores = np.array([]), np.array([]), np.array([])
    
    for qid, _ in tqdm(test_queries.values):
        
        query = qid_query_dict[qid]
        qids_i, pids_i, scores_i = np.array([]), np.array([]), np.array([])
        
        for pid in candidate_passages[candidate_passages['qid']==qid]['pid']:
            
            passage = pid_passage_dict[pid]
            
            # document length
            dl = len(passage)
            
            # compute K
            K = k1 * ((1-b) + b*dl/avdl)
            
            score = 0
            for term in query:
                qfi = query.count(term)
                if term in passage:
                    fi = p_inverted_index[term][pid][0]
                    ni = len(inverted_index[term])
                else:
                    fi = 0
                    ni = 0
                score += np.log(((0+0.5)/(0-0+0.5))/((ni-0+0.5)/(N-ni-0+0+0.5)))*((k1+1)*fi/((K+fi)))*((k2+1)*qfi/(k2+qfi))
            
            qids_i = np.append(qids_i, qid)
            pids_i = np.append(pids_i, pid)
            scores_i = np.append(scores_i, score)

        descending_idx = np.argsort(scores_i)[::-1]
        qids_i = qids_i[descending_idx]
        pids_i = pids_i[descending_idx]
        scores_i = scores_i[descending_idx]

        qids = np.append(qids, qids_i[:top])
        pids = np.append(pids, pids_i[:top])
        scores = np.append(scores, scores_i[:top])

    df['qid'] = qids
    df['pid'] = pids
    df['score'] = scores
    
    df['qid'] = df['qid'].astype(int)
    df['pid'] = df['pid'].astype(int)
    
    return df

In [60]:
query_params = [test_queries, qid_query_dict, query_inverted_index]
passage_params = [candidate_passages, pid_passage_dict, inverted_index]
model_params = [1.2, 100, 0.75]

In [61]:
df_bm25 = BM25_model(query_params, passage_params, model_params, top=100)

100%|█████████████████████████████████████████| 200/200 [00:03<00:00, 56.49it/s]


In [62]:
df_bm25.head()

,qid,pid,score
0,1108939,2846462,20.003898
1,1108939,7000936,19.966412
2,1108939,7724054,19.734075
3,1108939,5291621,19.535797
4,1108939,7651010,18.958465


In [57]:
len(df_bm25)

19290

In [58]:
df_bm25.to_csv('bm25.csv', header=False, sep=',', index=False)